# Tweeter Calculations - Scan-Speak D2608/913000

This notebook contains acoustic calculations for the Scan-Speak D2608/913000 1" dome tweeter in a sealed enclosure configuration.

## Driver Specifications
- **Model**: D2608/913000 (Discovery Series)
- **Type**: 1" textile soft dome tweeter
- **Impedance**: 8Ω
- **Sensitivity**: 91.3 dB @ 2.83V/1m
- **Fs**: 700 Hz
- **Usable Range**: 1,500 - 20,000+ Hz
- **Recommended Crossover**: ≥1,500 Hz

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import signal
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

ModuleNotFoundError: No module named 'numpy'

## 1. Driver Parameters

In [ ]:
# Scan-Speak D2608/913000 Parameters from specs.MD and datasheet
tweeter_params = {
    'name': 'Scan-Speak D2608/913000',
    'type': '1" Dome Tweeter',
    'series': 'Discovery',
    'also_known_as': 'Peerless HDS 810921',
    'impedance': 8,  # Ω (FROM SPECS.MD)
    'sensitivity': 91.3,  # dB @ 2.83V/1m (FROM SPECS.MD)
    'fs': 700,  # Hz - Free air resonance (FROM SPECS.MD)
    'bl': 2.6,  # Tm - BL product (FROM SPECS.MD)
    'cms': 0.29e-3,  # m/N - Compliance (FROM SPECS.MD: 0.29 mm/N)
    'mms': 0.18e-3,  # kg - Moving mass (FROM SPECS.MD: 0.18g)
    'xmax': 0.5e-3,  # m - Maximum linear excursion (FROM SPECS.MD)
    'sd': 7e-4,  # m² - Effective piston area (FROM SPECS.MD: 7 cm²)
    'vas': 0.0007 * 28.317,  # L - Convert ft³ to L (FROM SPECS.MD: 0.0007 ft³)
    'dome_diameter': 26e-3,  # m - Dome diameter (FROM SPECS.MD)
    'freq_range': (1500, 20000),  # Hz - Usable frequency range (FROM SPECS.MD)
    'crossover_min': 1500,  # Hz - Minimum recommended crossover (FROM SPECS.MD)
    'usage_range': (2750, 20000),  # Hz - Usage range in 3-way system
    'enclosure_type': 'sealed',
    'features': [
        'Very low mass textile soft dome diaphragm',
        'Optimized magnet system with double magnets', 
        'Low damping magnetic fluid (ferrofluid) cooling',
        'Fully vented motor system for low compression',
        '8 ohm impedance, positive terminal marked with white paint',
        'Replaceable voice coil assembly',
        'Black die-cast aluminum face plate',
        'Made in Denmark'
    ]
}

# Load real measurement data
try:
    fr_data = pd.read_csv('/home/tom/git/Speaker-Design/speakers/ScanSpeak D20608-913000/Frequency Response.csv', 
                         header=None, names=['freq', 'spl'])
    impedance_data = pd.read_csv('/home/tom/git/Speaker-Design/speakers/ScanSpeak D20608-913000/Impedance.csv', 
                               header=None, names=['freq', 'impedance'])
    
    print("✅ Real measurement data loaded successfully!")
    print(f"Frequency response: {len(fr_data)} points from {fr_data['freq'].min():.0f} to {fr_data['freq'].max():.0f} Hz")
    print(f"Impedance: {len(impedance_data)} points from {impedance_data['freq'].min():.0f} to {impedance_data['freq'].max():.0f} Hz")
    
    real_data_available = True
except FileNotFoundError:
    print("❌ Real measurement data not found")
    real_data_available = False

# Display parameters
df_params = pd.DataFrame([
    ['Driver Model', tweeter_params['name']],
    ['Type', tweeter_params['type']],
    ['Series', tweeter_params['series']],
    ['Also Known As', tweeter_params['also_known_as']],
    ['Impedance', f"{tweeter_params['impedance']} Ω"],
    ['Sensitivity', f"{tweeter_params['sensitivity']} dB @ 2.83V/1m"],
    ['Free Air Resonance (Fs)', f"{tweeter_params['fs']} Hz"],
    ['BL Product', f"{tweeter_params['bl']} Tm"],
    ['Compliance (Cms)', f"{tweeter_params['cms']*1000:.2f} mm/N"],
    ['Moving Mass (Mms)', f"{tweeter_params['mms']*1000:.2f} g"],
    ['Max Excursion (Xmax)', f"{tweeter_params['xmax']*1000:.1f} mm"],
    ['Effective Area (Sd)', f"{tweeter_params['sd']*1e4:.0f} cm²"],
    ['Equivalent Volume (Vas)', f"{tweeter_params['vas']:.4f} L"],
    ['Dome Diameter', f"{tweeter_params['dome_diameter']*1000:.0f} mm"],
    ['Usable Frequency Range', f"{tweeter_params['freq_range'][0]:,}-{tweeter_params['freq_range'][1]:,} Hz"],
    ['Min Crossover Frequency', f"{tweeter_params['crossover_min']:,} Hz"],
    ['Usage Range (3-way)', f"{tweeter_params['usage_range'][0]:,}-{tweeter_params['usage_range'][1]:,} Hz"],
    ['Enclosure Type', tweeter_params['enclosure_type']]
], columns=['Parameter', 'Value'])

print("\n📋 Scan-Speak D2608/913000 Parameters (Updated from specs.MD):")
print("=" * 70)
print(df_params.to_string(index=False))

print(f"\n🔧 KEY FEATURES:")
for feature in tweeter_params['features']:
    print(f"   • {feature}")

print(f"\n⚠️ PARAMETER UPDATES FROM SPECS.MD:")
print(f"   • Sensitivity: Updated to {tweeter_params['sensitivity']} dB (authoritative)")
print(f"   • All T-S parameters now match specs.MD")
print(f"   • Added complete physical specifications")
print(f"   • Confirmed Danish manufacture (high quality)")

print(f"\n🎯 TWEETER DESIGN TARGETS:")
print(f"   • Frequency range: {tweeter_params['usage_range'][0]:,}-{tweeter_params['usage_range'][1]:,} Hz")
print(f"   • High sensitivity: {tweeter_params['sensitivity']} dB")
print(f"   • Professional grade driver from Denmark")
print(f"   • Good impedance match: {tweeter_params['impedance']}Ω → 25W available from amp")
print(f"   • Safe crossover at {tweeter_params['usage_range'][0]:,} Hz (well above {tweeter_params['crossover_min']:,} Hz minimum)")

## 2. Frequency Response Analysis

In [ ]:
# Load real measurement data for Scan-Speak D2608/913000
import pandas as pd

# Read real frequency response data
try:
    fr_data = pd.read_csv('/home/tom/git/Speaker-Design/speakers/ScanSpeak D20608-913000/Frequency Response.csv', 
                         header=None, names=['freq', 'spl'])
    impedance_data = pd.read_csv('/home/tom/git/Speaker-Design/speakers/ScanSpeak D20608-913000/Impedance.csv', 
                               header=None, names=['freq', 'impedance'])
    
    print("✅ Real measurement data loaded successfully!")
    print(f"Frequency response: {len(fr_data)} points from {fr_data['freq'].min():.0f} to {fr_data['freq'].max():.0f} Hz")
    print(f"Impedance: {len(impedance_data)} points from {impedance_data['freq'].min():.0f} to {impedance_data['freq'].max():.0f} Hz")
    
    real_data_available = True
except FileNotFoundError:
    print("❌ Real measurement data not found - using theoretical model")
    real_data_available = False

# Define frequency range for analysis
freq = np.logspace(2, 5, 1000)  # 100 Hz to 100 kHz

def tweeter_response_theoretical(f, fs, sensitivity):
    """
    SIMPLIFIED tweeter response model - THEORETICAL ESTIMATE
    - Flat response above fs with gradual rolloff at high frequencies
    - 12 dB/octave rolloff below fs
    """
    response = np.zeros_like(f)
    
    # Below resonance: 12 dB/octave rolloff
    below_fs = f < fs
    response[below_fs] = sensitivity + 20 * np.log10(f[below_fs] / fs)
    
    # Above resonance: relatively flat with gentle high-frequency rolloff
    above_fs = f >= fs
    # Gentle rolloff starting around 10 kHz for textile dome
    rolloff_freq = 10000
    response[above_fs] = sensitivity - 6 * np.log10(1 + (f[above_fs] / rolloff_freq)**2)
    
    return response

# Plot frequency response
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))

if real_data_available:
    # Plot real measurement data
    ax1.semilogx(fr_data['freq'], fr_data['spl'], 'r-', linewidth=2, 
                label='Real Measurement Data', alpha=0.8)
    
    # Calculate theoretical for comparison
    tweeter_spl_theoretical = tweeter_response_theoretical(freq, tweeter_params['fs'], tweeter_params['sensitivity'])
    ax1.semilogx(freq, tweeter_spl_theoretical, 'b--', linewidth=1, alpha=0.6,
                label='Theoretical Model (for comparison)')
    
    title_suffix = "(REAL MEASUREMENT DATA)"
    data_label = "Real Measurement"
else:
    # Use theoretical model
    tweeter_spl = tweeter_response_theoretical(freq, tweeter_params['fs'], tweeter_params['sensitivity'])
    ax1.semilogx(freq, tweeter_spl, 'b-', linewidth=2, label='Tweeter Response (ESTIMATED)')
    title_suffix = "(THEORETICAL MODEL)"
    data_label = "Estimated"

# Add reference lines
ax1.axvline(tweeter_params['fs'], color='r', linestyle='--', alpha=0.7, label=f'Fs = {tweeter_params["fs"]} Hz')
ax1.axvline(tweeter_params['recommended_crossover'], color='g', linestyle='--', alpha=0.7, 
           label=f'Min Crossover = {tweeter_params["recommended_crossover"]} Hz')
ax1.axvspan(tweeter_params['usable_range'][0], tweeter_params['usable_range'][1], 
           alpha=0.2, color='green', label='Usable Range')

ax1.set_xlabel('Frequency (Hz)')
ax1.set_ylabel('SPL (dB)')
ax1.set_title(f'Scan-Speak D2608/913000 Frequency Response {title_suffix}')
ax1.grid(True, alpha=0.3)
ax1.legend()
ax1.set_xlim(100, 50000)

if real_data_available:
    ax1.set_ylim(fr_data['spl'].min() - 5, fr_data['spl'].max() + 5)
else:
    ax1.set_ylim(60, 100)
    # Add warning text for theoretical model
    ax1.text(0.02, 0.98, 'WARNING: Estimated curves - real measurement data needed!', 
             transform=ax1.transAxes, fontsize=10, fontweight='bold', 
             bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.8),
             verticalalignment='top')

# Impedance plot
if real_data_available:
    # Plot real impedance data
    ax2.semilogx(impedance_data['freq'], impedance_data['impedance'], 'r-', linewidth=2,
                label='Real Measurement Data', alpha=0.8)
    
    # Calculate theoretical for comparison
    Z_theoretical = tweeter_params['impedance'] * (1 + 2 * np.exp(-((freq - tweeter_params['fs'])/200)**2))
    ax2.semilogx(freq, Z_theoretical, 'b--', linewidth=1, alpha=0.6,
                label='Theoretical Model (for comparison)')
    
    impedance_title = "Real Impedance Measurement"
else:
    # Use theoretical model
    Z_tweeter = tweeter_params['impedance'] * (1 + 2 * np.exp(-((freq - tweeter_params['fs'])/200)**2))
    ax2.semilogx(freq, Z_tweeter, 'r-', linewidth=2, label='Impedance (ESTIMATED)')
    impedance_title = "Estimated Impedance Response (THEORETICAL MODEL)"
    
    # Add warning text for theoretical model
    ax2.text(0.02, 0.98, 'WARNING: Estimated curves - real measurement data needed!', 
             transform=ax2.transAxes, fontsize=10, fontweight='bold', 
             bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.8),
             verticalalignment='top')

ax2.axvline(tweeter_params['fs'], color='r', linestyle='--', alpha=0.7, label=f'Fs = {tweeter_params["fs"]} Hz')
ax2.set_xlabel('Frequency (Hz)')
ax2.set_ylabel('Impedance (Ω)')
ax2.set_title(impedance_title)
ax2.grid(True, alpha=0.3)
ax2.legend()
ax2.set_xlim(100, 50000)

if real_data_available:
    ax2.set_ylim(impedance_data['impedance'].min() - 1, impedance_data['impedance'].max() + 2)
else:
    ax2.set_ylim(4, 20)

plt.tight_layout()
plt.show()

# Print data analysis
if real_data_available:
    print(f"\n📊 REAL MEASUREMENT ANALYSIS:")
    print(f"   • Sensitivity range: {fr_data['spl'].min():.1f} to {fr_data['spl'].max():.1f} dB")
    print(f"   • Average SPL (1-10kHz): {fr_data[(fr_data['freq'] >= 1000) & (fr_data['freq'] <= 10000)]['spl'].mean():.1f} dB")
    print(f"   • Impedance range: {impedance_data['impedance'].min():.1f} to {impedance_data['impedance'].max():.1f} Ω")
    print(f"   • Nominal impedance: ~{impedance_data[(impedance_data['freq'] >= 1000) & (impedance_data['freq'] <= 10000)]['impedance'].mean():.1f} Ω")
    
    # Find resonance frequency from impedance peak
    resonance_idx = impedance_data['impedance'].idxmax()
    actual_fs = impedance_data.loc[resonance_idx, 'freq']
    print(f"   • Measured Fs (impedance peak): {actual_fs:.0f} Hz")
    print(f"   • Spec Fs vs Measured: {tweeter_params['fs']} Hz vs {actual_fs:.0f} Hz")
else:
    print(f"\n⚠️ Using theoretical model - real measurements would provide:")
    print(f"   • Actual frequency response characteristics")
    print(f"   • True resonance frequency")
    print(f"   • Real impedance behavior")
    print(f"   • Breakup modes and irregularities")

## 3. Crossover Design

In [ ]:
# Crossover parameters
crossover_freq = 2750  # Hz - Mid to tweeter crossover from specs
crossover_order = 4  # 4th order = 24 dB/octave (Linkwitz-Riley)

def linkwitz_riley_highpass(f, fc, order=4):
    """
    Linkwitz-Riley high-pass filter response
    """
    s = 1j * 2 * np.pi * f / (2 * np.pi * fc)
    
    if order == 4:
        # 4th order LR = cascaded 2nd order Butterworth
        h = s**2 / (s**2 + np.sqrt(2)*s + 1)
        return 20 * np.log10(np.abs(h)**2)
    else:
        # General case
        h = (s**(order//2)) / ((s**(order//2)) + 1)
        return 20 * np.log10(np.abs(h))

# Calculate crossover response
crossover_response = linkwitz_riley_highpass(freq, crossover_freq, crossover_order)

# Get tweeter response for crossover analysis
if real_data_available:
    # Use real measurement data for crossover analysis
    # Interpolate real data to match frequency array
    from scipy.interpolate import interp1d
    
    # Create interpolation function for real data
    freq_interp = interp1d(fr_data['freq'], fr_data['spl'], 
                          bounds_error=False, fill_value='extrapolate')
    tweeter_spl_for_crossover = freq_interp(freq)
else:
    # Use theoretical model
    tweeter_spl_for_crossover = tweeter_response_theoretical(freq, tweeter_params['fs'], tweeter_params['sensitivity'])

# Combined response (tweeter + crossover)
tweeter_filtered = tweeter_spl_for_crossover + crossover_response

# Plot crossover analysis
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))

# Crossover filter response
ax1.semilogx(freq, crossover_response, 'g-', linewidth=2, label=f'{crossover_order}th Order LR High-Pass')
ax1.axvline(crossover_freq, color='r', linestyle='--', alpha=0.7, 
           label=f'Crossover = {crossover_freq} Hz')
ax1.axhline(-3, color='k', linestyle=':', alpha=0.5, label='-3 dB')
ax1.axhline(-6, color='k', linestyle=':', alpha=0.5, label='-6 dB')
ax1.set_xlabel('Frequency (Hz)')
ax1.set_ylabel('Magnitude (dB)')
ax1.set_title(f'Tweeter Crossover Filter Response ({crossover_freq} Hz, {crossover_order}th Order LR)')
ax1.grid(True, alpha=0.3)
ax1.legend()
ax1.set_xlim(1000, 20000)
ax1.set_ylim(-60, 5)

# Combined response
data_label = "Real Measurement" if real_data_available else "Theoretical Model"
line_style = 'r-' if real_data_available else 'b--'

ax2.semilogx(freq, tweeter_spl_for_crossover, line_style, alpha=0.7, 
            label=f'Raw Tweeter Response ({data_label})')
ax2.semilogx(freq, tweeter_filtered, 'k-', linewidth=2, 
            label='Tweeter + Crossover')
ax2.axvline(crossover_freq, color='r', linestyle='--', alpha=0.7, 
           label=f'Crossover = {crossover_freq} Hz')
ax2.set_xlabel('Frequency (Hz)')
ax2.set_ylabel('SPL (dB)')
ax2.set_title(f'Tweeter Response with Crossover Filter ({data_label})')
ax2.grid(True, alpha=0.3)
ax2.legend()
ax2.set_xlim(1000, 20000)

if real_data_available:
    ax2.set_ylim(60, 100)
else:
    ax2.set_ylim(60, 95)

plt.tight_layout()
plt.show()

# Print crossover analysis
print(f"\n🎛️ CROSSOVER ANALYSIS:")
print(f"=" * 50)
print(f"Crossover Frequency: {crossover_freq} Hz")
print(f"Crossover Order: {crossover_order}th order (24 dB/octave)")
print(f"Filter Type: Linkwitz-Riley")
print(f"Safety Margin above Fs: {crossover_freq/tweeter_params['fs']:.1f}x")
print(f"Well within usable range: {crossover_freq >= tweeter_params['recommended_crossover']}")

if real_data_available:
    print(f"Data source: Real measurement data")
    # Calculate average response in crossover region
    crossover_mask = (fr_data['freq'] >= crossover_freq*0.8) & (fr_data['freq'] <= crossover_freq*1.2)
    if crossover_mask.any():
        avg_response = fr_data[crossover_mask]['spl'].mean()
        print(f"Average SPL near crossover: {avg_response:.1f} dB")
else:
    print(f"Data source: Theoretical model")
    print(f"⚠️ Real measurements would provide more accurate crossover design")

## 4. Power Handling and Thermal Analysis

In [ ]:
# Power analysis - Updated with CORRECTED OFFICIAL power ratings
amplifier_power = 25  # W @ 8Ω (from XH-M180 specs)

# CORRECTED power ratings from datasheet (the "80400W" appears to be a formatting error)
# Based on typical dome tweeter ratings and the "W" on second line suggesting missing value
tweeter_power_100h = 80  # W - 100h RMS noise test (IEC 17.1) - ASSUME this is correct
tweeter_power_long_term = 40  # W - Long-term max power (IEC 17.3) - ASSUME based on typical ratios

print("⚠️ DATASHEET POWER RATING CLARIFICATION NEEDED:")
print("Original datasheet shows:")
print("  100h RMS noise test (IEC 17.1): 80400W  ← LIKELY FORMATTING ERROR")
print("  Long-term max power (IEC 17.3): W       ← MISSING VALUE")
print("")
print("ASSUMPTIONS USED (need verification):")
print(f"  100h RMS: {tweeter_power_100h}W (assuming 80W, ignoring '400')")
print(f"  Long-term: {tweeter_power_long_term}W (typical 50% of 100h rating)")
print("  → RECOMMEND: Verify actual power ratings with manufacturer!")

typical_power_usage = 8  # W - estimated typical usage
peak_power_usage = 15  # W - estimated peak usage

def calculate_spl_from_power(power_watts, sensitivity_db, distance_m=1):
    """
    Calculate SPL from power, sensitivity, and distance
    """
    # Convert power to dBW
    power_dbw = 10 * np.log10(power_watts)
    
    # Calculate SPL at 1m
    spl_1m = sensitivity_db + power_dbw
    
    # Adjust for distance (6 dB per doubling of distance)
    if distance_m != 1:
        spl = spl_1m - 20 * np.log10(distance_m)
    else:
        spl = spl_1m
    
    return spl

# Calculate SPL levels - using CONSERVATIVE power estimates
power_levels = np.array([1, 2, 5, 8, 10, 15, 20, 25, 40, 80])  # Watts
spl_levels = [calculate_spl_from_power(p, tweeter_params['sensitivity']) for p in power_levels]

# Create power vs SPL table
power_table = pd.DataFrame({
    'Power (W)': power_levels,
    'SPL @ 1m (dB)': [f"{spl:.1f}" for spl in spl_levels],
    'SPL @ 2m (dB)': [f"{calculate_spl_from_power(p, tweeter_params['sensitivity'], 2):.1f}" for p in power_levels],
    'SPL @ 3m (dB)': [f"{calculate_spl_from_power(p, tweeter_params['sensitivity'], 3):.1f}" for p in power_levels],
    'Status': ['Low', 'Low', 'Moderate', 'Typical', 'Safe', 'Peak Est', 'Conservative', 'Amp Max', 'Est LT Max', 'Est 100h Max']
})

print(f"\nPower vs SPL Analysis (CONSERVATIVE estimates - verify actual ratings!):")
print("=" * 70)
print(power_table.to_string(index=False))

# Plot power vs SPL
plt.figure(figsize=(12, 8))
plt.plot(power_levels, spl_levels, 'b-o', linewidth=2, markersize=6, label='SPL @ 1m')
plt.plot(power_levels, [calculate_spl_from_power(p, tweeter_params['sensitivity'], 2) for p in power_levels], 
         'g-s', linewidth=2, markersize=6, label='SPL @ 2m')
plt.plot(power_levels, [calculate_spl_from_power(p, tweeter_params['sensitivity'], 3) for p in power_levels], 
         'r-^', linewidth=2, markersize=6, label='SPL @ 3m')

# Add power limit lines
plt.axvline(typical_power_usage, color='orange', linestyle='--', alpha=0.7, label=f'Typical Usage ({typical_power_usage}W)')
plt.axvline(amplifier_power, color='red', linestyle='--', alpha=0.7, label=f'Amplifier Limit ({amplifier_power}W)')
plt.axvline(tweeter_power_long_term, color='purple', linestyle=':', alpha=0.5, 
           label=f'Est. Tweeter Long-term ({tweeter_power_long_term}W)')
plt.axvline(tweeter_power_100h, color='darkred', linestyle=':', alpha=0.5, 
           label=f'Est. Tweeter 100h ({tweeter_power_100h}W)')

plt.xlabel('Power (W)')
plt.ylabel('SPL (dB)')
plt.title('Tweeter Power vs SPL Relationship (VERIFY POWER RATINGS!)')
plt.grid(True, alpha=0.3)
plt.legend()
plt.xlim(0, 85)
plt.ylim(85, 130)

# Add warning annotation
plt.text(0.02, 0.98, 'WARNING: Power ratings need verification!\nDatasheet shows unclear values', 
         transform=plt.gca().transAxes, fontsize=10, fontweight='bold', 
         bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.8),
         verticalalignment='top')

plt.show()

print(f"\n⚡ POWER ANALYSIS SUMMARY (NEEDS VERIFICATION):")
print(f"=" * 60)
print(f"Available amplifier power: {amplifier_power}W @ 8Ω")
print(f"Estimated tweeter long-term max: {tweeter_power_long_term}W (VERIFY!)")
print(f"Estimated tweeter 100h RMS max: {tweeter_power_100h}W (VERIFY!)")
print(f"Recommended filter: {tweeter_params['filter_rec']}")
print(f"")
print(f"Power Usage Scenarios:")
print(f"• Typical usage: {typical_power_usage}W → {calculate_spl_from_power(typical_power_usage, tweeter_params['sensitivity']):.1f} dB @ 1m")
print(f"• Peak usage: {peak_power_usage}W → {calculate_spl_from_power(peak_power_usage, tweeter_params['sensitivity']):.1f} dB @ 1m")
print(f"• Amplifier limit: {amplifier_power}W → {calculate_spl_from_power(amplifier_power, tweeter_params['sensitivity']):.1f} dB @ 1m")
print(f"• Est. tweeter long-term: {tweeter_power_long_term}W → {calculate_spl_from_power(tweeter_power_long_term, tweeter_params['sensitivity']):.1f} dB @ 1m")
print(f"")
print(f"Safety Analysis (based on estimates):")
print(f"• Amplifier vs Est. Tweeter LT: {tweeter_power_long_term/amplifier_power:.1f}x safety factor")
print(f"• Typical vs Amplifier: {amplifier_power/typical_power_usage:.1f}x headroom")

# Check if amplifier is safe for tweeter (conservative approach)
if amplifier_power <= tweeter_power_long_term:
    print(f"\n✅ AMPLIFIER COMPATIBILITY: LIKELY SAFE (verify power ratings)")
    print(f"   Amplifier power ({amplifier_power}W) ≤ Est. tweeter long-term ({tweeter_power_long_term}W)")
else:
    print(f"\n⚠️ AMPLIFIER COMPATIBILITY: REQUIRES CAREFUL VERIFICATION")
    print(f"   Amplifier power ({amplifier_power}W) vs Est. tweeter long-term ({tweeter_power_long_term}W)")

print(f"\n🚨 ACTION REQUIRED:")
print(f"   1. Contact Scan-Speak to verify actual power ratings")
print(f"   2. Check if '80400W' is a typo (likely should be '80W' or '40W')")
print(f"   3. Confirm long-term power rating value")
print(f"   4. Update calculations once confirmed")

## 5. Enclosure Design

In [ ]:
# Tweeter enclosure analysis
# Tweeters typically need minimal sealed chamber for optimal performance

# Recommended enclosure volume (very small for tweeter)
enclosure_volume_L = 0.05  # Liters - minimal sealed chamber
enclosure_volume_m3 = enclosure_volume_L / 1000

# Calculate enclosure effects (minimal for tweeter)
def tweeter_enclosure_effects(vas, vb):
    """
    Calculate the effect of enclosure on tweeter performance
    For tweeters, this is minimal due to very small Vas
    """
    alpha = vas / vb  # Compliance ratio
    
    # Effects are minimal for tweeters
    qtc_multiplier = np.sqrt(1 + alpha)  # Very close to 1
    fc_multiplier = np.sqrt(1 + alpha)   # Very close to 1
    
    return qtc_multiplier, fc_multiplier, alpha

# Calculate enclosure effects
qtc_mult, fc_mult, alpha = tweeter_enclosure_effects(tweeter_params['vas'], enclosure_volume_m3)

# Physical dimensions for 3D printing
dome_diameter = tweeter_params['dome_diameter'] * 1000  # mm
cutout_diameter = dome_diameter + 5  # mm - typical clearance
mounting_depth = 20  # mm - typical tweeter depth requirement
chamber_depth = 30  # mm - sealed chamber behind tweeter

# Calculate chamber dimensions
chamber_volume_cm3 = enclosure_volume_L * 1000  # cm³
chamber_diameter = 60  # mm - reasonable chamber diameter
required_chamber_depth = (chamber_volume_cm3 * 4) / (np.pi * (chamber_diameter/10)**2)  # cm

print("\n=== TWEETER ENCLOSURE DESIGN ===")
print(f"\nDriver: {tweeter_params['name']}")
print(f"Dome Diameter: {dome_diameter:.0f} mm")
print(f"Recommended Cutout: {cutout_diameter:.0f} mm")
print(f"Mounting Depth Required: {mounting_depth} mm")

print(f"\nSealed Chamber Design:")
print(f"Recommended Volume: {enclosure_volume_L:.3f} L ({chamber_volume_cm3:.0f} cm³)")
print(f"Chamber Diameter: {chamber_diameter} mm")
print(f"Required Chamber Depth: {required_chamber_depth:.1f} cm")

print(f"\nEnclosure Effects (minimal for tweeter):")
print(f"Compliance Ratio (α): {alpha:.6f}")
print(f"Qtc Change: {qtc_mult:.4f}x (virtually no change)")
print(f"Fc Change: {fc_mult:.4f}x (virtually no change)")

print(f"\n3D Printing Specifications:")
print(f"- Minimal sealed chamber behind tweeter")
print(f"- Smooth internal surfaces")
print(f"- Gasket groove for airtight seal")
print(f"- Fill with acoustic foam (optional)")
print(f"- Wall thickness: 3-4mm minimum")

# Create a simple visualization of the tweeter enclosure
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Front view
circle1 = plt.Circle((0, 0), dome_diameter/2, fill=False, color='blue', linewidth=2, label='Dome')
circle2 = plt.Circle((0, 0), cutout_diameter/2, fill=False, color='red', linestyle='--', linewidth=2, label='Cutout')
ax1.add_patch(circle1)
ax1.add_patch(circle2)
ax1.set_xlim(-40, 40)
ax1.set_ylim(-40, 40)
ax1.set_aspect('equal')
ax1.set_title('Tweeter Front View')
ax1.grid(True, alpha=0.3)
ax1.legend()
ax1.set_xlabel('Distance (mm)')
ax1.set_ylabel('Distance (mm)')

# Side view cross-section
ax2.add_patch(plt.Rectangle((-chamber_diameter/2, -chamber_depth), chamber_diameter, chamber_depth, 
                           fill=False, edgecolor='blue', linewidth=2, label='Chamber'))
ax2.add_patch(plt.Rectangle((-dome_diameter/2, 0), dome_diameter, mounting_depth, 
                           fill=False, edgecolor='red', linewidth=2, label='Tweeter'))
ax2.set_xlim(-40, 40)
ax2.set_ylim(-40, 30)
ax2.set_title('Tweeter Side View (Cross-section)')
ax2.grid(True, alpha=0.3)
ax2.legend()
ax2.set_xlabel('Distance (mm)')
ax2.set_ylabel('Distance (mm)')

plt.tight_layout()
plt.show()

## 6. DSP Configuration Recommendations

In [ ]:
# DSP recommendations for tweeter
print("\n=== DSP CONFIGURATION FOR TWEETER ===")
print(f"\nCrossover Settings:")
print(f"- High-pass filter: {crossover_freq} Hz")
print(f"- Filter type: Linkwitz-Riley 24 dB/octave")
print(f"- Phase: 0° (LR filters have inherent phase alignment)")

print(f"\nEQ Recommendations:")
print(f"- Gentle high-frequency shelf (if needed): +1 to +2 dB above 8kHz")
print(f"- Room correction: Adjust based on measurements")
print(f"- Avoid boosting below crossover frequency")

print(f"\nProtection Settings:")
print(f"- Limiter threshold: {peak_power_usage}W equivalent ({calculate_spl_from_power(peak_power_usage, tweeter_params['sensitivity']):.1f} dB @ 1m)")
print(f"- High-frequency limiter: Fast attack, slow release")
print(f"- Hard limiter at amplifier power: {amplifier_power}W")

print(f"\nTime Alignment:")
print(f"- Tweeter is typically advanced (negative delay)")
print(f"- Typical delay: 0.1 - 0.5 ms behind midrange")
print(f"- Fine-tune with measurements")

print(f"\nGain Structure:")
print(f"- Input sensitivity: Adjust for desired SPL")
print(f"- Match tweeter level to midrange at crossover point")
print(f"- Typical tweeter attenuation: 0 to -3 dB")

# Create DSP signal flow diagram conceptually
dsp_stages = ['Input\nGain', 'Room\nCorrection\nEQ', 'High-Pass\nFilter\n2750 Hz', 'Time\nAlignment', 'Limiter', 'Output\nto Amp']

fig, ax = plt.subplots(figsize=(12, 4))
for i, stage in enumerate(dsp_stages):
    rect = plt.Rectangle((i*2, 0), 1.5, 1, fill=True, facecolor='lightblue', 
                        edgecolor='blue', linewidth=2)
    ax.add_patch(rect)
    ax.text(i*2 + 0.75, 0.5, stage, ha='center', va='center', fontsize=10, fontweight='bold')
    
    if i < len(dsp_stages) - 1:
        ax.arrow(i*2 + 1.5, 0.5, 0.4, 0, head_width=0.1, head_length=0.1, fc='red', ec='red')

ax.set_xlim(-0.5, len(dsp_stages)*2)
ax.set_ylim(-0.5, 1.5)
ax.set_title('DSP Signal Flow for Tweeter Channel', fontsize=14, fontweight='bold')
ax.axis('off')
plt.tight_layout()
plt.show()

## 7. Summary and Recommendations

In [ ]:
print("\n" + "="*60)
print(" TWEETER DESIGN SUMMARY - Scan-Speak D2608/913000")
print("="*60)

print(f"\n🔊 DRIVER SPECIFICATIONS:")
print(f"   • Type: 1\" textile soft dome tweeter")
print(f"   • Impedance: {tweeter_params['impedance']}Ω")
print(f"   • Sensitivity: {tweeter_params['sensitivity']} dB @ 2.83V/1m")
print(f"   • Usable range: {tweeter_params['usable_range'][0]}-{tweeter_params['usable_range'][1]:,} Hz")
print(f"   • Excellent quality Danish-made driver")

print(f"\n⚡ POWER HANDLING:")
print(f"   • Available power: {amplifier_power}W @ 8Ω")
print(f"   • Recommended usage: {typical_power_usage}W typical")
print(f"   • SPL capability: {calculate_spl_from_power(amplifier_power, tweeter_params['sensitivity']):.1f} dB @ 1m (max)")
print(f"   • Excellent power headroom: {amplifier_power/typical_power_usage:.1f}x")

print(f"\n🎛️ CROSSOVER DESIGN:")
print(f"   • Crossover frequency: {crossover_freq} Hz")
print(f"   • Filter type: 24 dB/octave Linkwitz-Riley")
print(f"   • Safe margin above Fs: {crossover_freq/tweeter_params['fs']:.1f}x")
print(f"   • Digital implementation via ADAU1701")

print(f"\n📦 ENCLOSURE REQUIREMENTS:")
print(f"   • Type: Minimal sealed chamber")
print(f"   • Volume: {enclosure_volume_L:.3f}L ({chamber_volume_cm3:.0f} cm³)")
print(f"   • Cutout diameter: {cutout_diameter:.0f}mm")
print(f"   • Chamber depth: ~{required_chamber_depth:.0f}cm")
print(f"   • Negligible enclosure effects")

print(f"\n🖨️ 3D PRINTING NOTES:")
print(f"   • Material: PETG/ABS/ASA recommended")
print(f"   • Wall thickness: 3-4mm minimum")
print(f"   • Smooth internal surfaces critical")
print(f"   • Include gasket groove for airtight seal")
print(f"   • Optional: Light acoustic foam fill")

print(f"\n⚙️ DSP CONFIGURATION:")
print(f"   • High-pass: {crossover_freq} Hz, LR4 (24 dB/oct)")
print(f"   • Time alignment: ~0.1-0.5ms delay vs midrange")
print(f"   • Protection: Limiter at {peak_power_usage}W")
print(f"   • EQ: Room correction + gentle HF shelf if needed")

# Check if real data was loaded
try:
    if real_data_available:
        print(f"\n✅ REAL MEASUREMENT DATA AVAILABLE:")
        print(f"   • Frequency response: ✓ ({len(fr_data)} measurement points)")
        print(f"   • Impedance curve: ✓ ({len(impedance_data)} measurement points)")
        print(f"   • Measured sensitivity range: {fr_data['spl'].min():.1f} to {fr_data['spl'].max():.1f} dB")
        
        # Find actual resonance from measurements
        resonance_idx = impedance_data['impedance'].idxmax()
        actual_fs = impedance_data.loc[resonance_idx, 'freq']
        print(f"   • Measured Fs: {actual_fs:.0f} Hz (vs {tweeter_params['fs']} Hz spec)")
        print(f"   • Real impedance range: {impedance_data['impedance'].min():.1f}-{impedance_data['impedance'].max():.1f} Ω")
        
        # Calculate average sensitivity in usable range
        usable_mask = (fr_data['freq'] >= tweeter_params['usable_range'][0]) & (fr_data['freq'] <= tweeter_params['usable_range'][1])
        avg_sensitivity = fr_data[usable_mask]['spl'].mean()
        print(f"   • Average sensitivity (1.5-20kHz): {avg_sensitivity:.1f} dB")
        
        data_status = "✅ EXCELLENT"
    else:
        print(f"\n❌ MEASUREMENT DATA LIMITATIONS:")
        data_status = "❌ CRITICAL LIMITATION"
except NameError:
    print(f"\n❌ MEASUREMENT DATA LIMITATIONS:")
    print(f"   • No real measurement data available for Scan-Speak D2608/913000")
    print(f"   • Frequency response curves are THEORETICAL ESTIMATES")
    print(f"   • Impedance curves are SIMPLIFIED MODELS")
    data_status = "❌ CRITICAL LIMITATION"

print(f"\n📊 DATA STATUS SUMMARY:")
print(f"   • Tang Band W3-1876S: ✅ (CSV + QSP files)")
print(f"   • Tang Band W3-871SC: ✅ (CSV + QSP files)")
print(f"   • Scan-Speak D2608/913000: {data_status}")

print(f"\n✅ DESIGN VALIDATION:")
print(f"   • Crossover well above resonance: ✓")
print(f"   • Adequate power handling: ✓")
print(f"   • Minimal enclosure effects: ✓")
print(f"   • High-quality driver selection: ✓")
print(f"   • Proper impedance matching: ✓")
try:
    if real_data_available:
        print(f"   • Real measurement data: ✅")
    else:
        print(f"   • Real measurement data: ❌")
except NameError:
    print(f"   • Real measurement data: ❌")

print(f"\n" + "="*60)
try:
    if real_data_available:
        print(" ✅ READY FOR PRECISE 3D MODELING AND DSP PROGRAMMING! ")
    else:
        print(" RECOMMENDATION: Obtain real measurements for tweeter! ")
except NameError:
    print(" RECOMMENDATION: Obtain real measurements for tweeter! ")
print("="*60)